---
description: Cookbook that demonstrates how to run Langchain evaluations on data in Langfuse.
category: Evaluation
---

# Run Langchain Evaluations on data in Langfuse

This cookbook shows how model-based evaluations can be used to automate the evaluation of production completions in Langfuse. This example uses Langchain and is adaptable to other libraries. Which library is the best to use depends heavily on the use case.

This cookbook follows three steps:
1. Fetch production `generations` stored in Langfuse
2. Evaluate these `generations` using Langchain
3. Ingest results back into Langfuse as `scores`


----
Not using Langfuse yet? [Get started](https://langfuse.com/docs/get-started) by capturing LLM events.

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

### Setup

First you need to install Langfuse and Langchain via pip and then set the environment variables.

In [2]:
%pip install langfuse langchain openai cohere tiktoken --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.1/135.1 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 10.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 24.0 MB/s eta 0:00:00
  Using cached tiktoken-0.6.0-cp310-cp310-macosx_11_0_arm64.whl (949 kB)
  Using cached chevron-0.14.0-py3-none-any.whl (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.5 MB/s eta 0:00:00a 0:00:01
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 33.8 MB/s eta 0:00:00
  Using cached orjson-3.9.15-cp310-cp310-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl (248 kB)
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.3
    Uninstalling

In [26]:
import os
os.environ['EVAL_MODEL'] = "text-davinci-003"
# os.environ['EVAL_MODEL'] = "gpt-3.5-turbo"

# Langchain Eval types
EVAL_TYPES={
    "hallucination": True,
    "conciseness": True,
    "relevance": True,
    "coherence": True,
    "harmfulness": True,
    "maliciousness": True,
    "helpfulness": True,
    "controversiality": True,
    "misogyny": True,
    "criminality": True,
    "insensitivity": True
}

Initialize the Langfuse Python SDK, more information [here](https://langfuse.com/docs/sdk/python#1-installation).

In [27]:
from langfuse import Langfuse

langfuse = Langfuse()

langfuse.auth_check()

True

### Fetching data

Load all `generations` from Langfuse filtered by `name`, in this case `OpenAI`. Names are used in Langfuse to identify different types of generations within an application. Change it to the name you want to evaluate.

Checkout [docs](https://langfuse.com/docs/sdk/python#generation) on how to set the name when ingesting an LLM Generation.

In [28]:
def fetch_all_pages(name=None, user_id = None, limit=50):
    page = 1
    all_data = []

    while True:
        response = langfuse.get_generations(name=name, limit=limit, user_id=user_id, page=page)
        if not response.data:
            break

        all_data.extend(response.data)
        page += 1

    return all_data

In [29]:
# generations = fetch_all_pages(user_id='user:abc')
generations = fetch_all_pages()
print(generations)

[ObservationsView(id='7fe0f7c8-4ee8-4dbd-a210-679cd7ca98f5', trace_id='c86e90c4-e601-4249-876f-e1246088fe8c', type='GENERATION', name='poet', start_time=datetime.datetime(2024, 3, 25, 19, 26, 59, 317000, tzinfo=datetime.timezone.utc), end_time=datetime.datetime(2024, 3, 25, 19, 27, 3, 120000, tzinfo=datetime.timezone.utc), completion_start_time=None, model='gpt-3.5-turbo-0125', model_parameters={'top_p': 1, 'max_tokens': 200, 'temperature': 1, 'presence_penalty': 0, 'frequency_penalty': 0}, input=[{'role': 'system', 'content': 'You are a poet. Create a poem about a city.'}, {'role': 'user', 'content': 'Sofia'}], version=None, metadata=None, output={'role': 'assistant', 'content': "In the heart of the Balkans, where history meets modernity,\nLies a city of beauty, Sofia, a place of pure serenity.\nWith ancient ruins whispering tales of days long gone,\nAnd vibrant street art that dances with the dawn.\n\nA melting pot of cultures, where East meets West,\nSofia's charm will put your wand

### Set up evaluation functions

In this section, we define functions to set up the Langchain eval based on the entries in `EVAL_TYPES`. Hallucinations require their own function. More on the Langchain evals can be found [here](https://python.langchain.com/docs/guides/evaluation/string/criteria_eval_chain).

In [30]:
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.evaluation.criteria import LabeledCriteriaEvalChain

def get_evaluator_for_key(key: str):
  llm = OpenAI(temperature=0, model=os.environ.get('EVAL_MODEL'))
  return load_evaluator("criteria", criteria=key, llm=llm)

def get_hallucination_eval():
  criteria = {
    "hallucination": (
      "Does this submission contain information"
      " not present in the input or reference?"
    ),
  }
  llm = OpenAI(temperature=0, model=os.environ.get('EVAL_MODEL'))

  return LabeledCriteriaEvalChain.from_llm(
      llm=llm,
      criteria=criteria,
  )

### Execute evaluation

Below, we execute the evaluation for each `Generation` loaded above. Each score is ingested into Langfuse via [`langfuse.score()`](https://langfuse.com/docs/scores).


In [31]:
def execute_eval_and_score():

  for generation in generations:
    criteria = [key for key, value in EVAL_TYPES.items() if value and key != "hallucination"]

    for criterion in criteria:
      eval_result = get_evaluator_for_key(criterion).evaluate_strings(
          prediction=generation.output,
          input=generation.input,
      )
      print(eval_result)

      langfuse.score(name=criterion, trace_id=generation.trace_id, observation_id=generation.id, value=eval_result["score"], comment=eval_result['reasoning'])

execute_eval_and_score()


NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [11]:
# hallucination


def eval_hallucination():

  chain = get_hallucination_eval()

  for generation in generations:
    eval_result = chain.evaluate_strings(
      prediction=generation.output,
      input=generation.input,
      reference=generation.input
    )
    print(eval_result)
    if eval_result is not None and eval_result["score"] is not None and eval_result["reasoning"] is not None:
      langfuse.score(name='hallucination', trace_id=generation.trace_id, observation_id=generation.id, value=eval_result["score"], comment=eval_result['reasoning'])


In [14]:
if EVAL_TYPES.get("hallucination") == True:
  eval_hallucination()

In [15]:
# SDK is async, make sure to await all requests
langfuse.flush()

### See Scores in Langfuse

 In the Langfuse UI, you can filter Traces by `Scores` and look into the details for each. Check out Langfuse Analytics to understand the impact of new prompt versions or application releases on these scores.

![Image of Trace](https://langfuse.com/images/docs/trace-conciseness-score.jpg)
_Example trace with conciseness score_


## Get in touch

Looking for a specific way to score your production data in Langfuse? Join the [Discord](https://langfuse.com/discord) and discuss your use case!